# Two layer net

## 背景

构建一个两层神经网络

## 准备工作

1.创建一个SBT项目，并引入相关依赖（参照[Getting Started](https://github.com/ThoughtWorksInc/DeepLearning.scala/wiki/Getting-Started) 或者将下面的依赖引入build.sbt, 注意DeepLearning.scala暂不支持scala2.12.X )
```
libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiableany" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiablenothing" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiableseq" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiabledouble" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiablefloat" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiablehlist" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiablecoproduct" % "latest.release"

libraryDependencies += "com.thoughtworks.deeplearning" %% "differentiableindarray" % "latest.release"

addCompilerPlugin("com.thoughtworks.implicit-dependent-type" %% "implicit-dependent-type" % "latest.release")

addCompilerPlugin("org.scalamacros" % "paradise" % "2.1.0" cross CrossVersion.full)

fork := true
```
2.[下载CIFAR-10 binary version (suitable for C programs)](https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz)，文件大小162 MB，md5sum：c32a1d4ab5d03f1284b67883e8d87530

3.将下载好的文件解压到src/main/resources目录。

4.Scala类ReadCIFAR10ToNDArray用于从上面的文件中读取图片及其标签数据并做归一化处理（[更多信息](https://www.cs.toronto.edu/~kriz/cifar.html)）：

In [1]:
import $plugin.$ivy.`com.thoughtworks.implicit-dependent-type::implicit-dependent-type:1.0.0`

import $ivy.`com.thoughtworks.deeplearning::differentiableany:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiablenothing:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiableseq:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiabledouble:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiablefloat:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiablehlist:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiablecoproduct:1.0.0-RC5`
import $ivy.`com.thoughtworks.deeplearning::differentiableindarray:1.0.0-RC5`

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.0`

import java.io.{FileInputStream, InputStream}


import com.thoughtworks.deeplearning
import org.nd4j.linalg.api.ndarray.INDArray
import com.thoughtworks.deeplearning.DifferentiableHList._
import com.thoughtworks.deeplearning.DifferentiableDouble._
import com.thoughtworks.deeplearning.DifferentiableINDArray._
import com.thoughtworks.deeplearning.DifferentiableAny._
import com.thoughtworks.deeplearning.DifferentiableINDArray.Optimizers._
import com.thoughtworks.deeplearning.{DifferentiableHList, DifferentiableINDArray, Layer}
import com.thoughtworks.deeplearning.Layer.Batch
import com.thoughtworks.deeplearning.Lift.Layers.Identity
import com.thoughtworks.deeplearning.Lift._
import com.thoughtworks.deeplearning.Poly.MathFunctions._
import com.thoughtworks.deeplearning.Poly.MathMethods./
import com.thoughtworks.deeplearning.Poly.MathOps
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.cpu.nativecpu.NDArray
import org.nd4j.linalg.factory.Nd4j
import org.nd4j.linalg.indexing.{INDArrayIndex, NDArrayIndex}
import org.nd4j.linalg.ops.transforms.Transforms
import org.nd4s.Implicits._
import shapeless._

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

import scala.collection.immutable.IndexedSeq

pprintConfig() = pprintConfig().copy(height = 5)//减少输出的行数，避免页面输出太长

import $file.ReadCIFAR10ToNDArray,ReadCIFAR10ToNDArray._

import $plugin.$                                                                             


import $ivy.$                                                           

import $ivy.$                                                               

import $ivy.$                                                           

import $ivy.$                                                              

import $ivy.$                                                             

import $ivy.$                                                             

import $ivy.$                                                                 

import $ivy.$                                                                


import $ivy.$                                             


import java.io.{FileInputStream, InputStream}



import com.thoughtworks.deeplearning

import org.nd4j.linalg.api.ndarray.INDArray

import com.thoughtworks.deeplearning.DifferentiableHList._

import com.thoughtworks.deeplearning

5.[Mini-Batch Gradient Descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent): 在大规模数据训练时，数据可以达到百万级量级。如果计算整个训练集，来获得仅仅一个参数的更新速度就太慢了。一个常用的方法是计算训练集中的小批量（batches）数据以提升参数更新速度。

   
6.如果你使用IntelliJ或者eclipse等其它IDE，智能提示可能会失效，代码有部分可能会爆红，这是IDE的问题，代码本身并无问题。
   

## 构建神经网络

1.新建一个Scala类TwoLayerNet

2.从CIFAR10 database中读取测试数据的图片和标签信息

In [2]:
//CIFAR10中的图片共有10个分类(airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck)
val CLASSES: Int = 10

//加载测试数据，我们读取100条作为测试数据
val testNDArray =
  ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/test_batch.bin", 100)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


NumberOfClasses: Int = 10
testNDArray: INDArray :: INDArray :: HNil = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0.55, 0.55, 0.56, 0.54, 0.49, 0.45, 0.59, 0.59, 0.62, 0.65, 0.63, 0.62, 0.64, 0.63, 0.64, 0.61, 0.61, 0.62, 0.64, 0.66, 0.67, 0.67, 0.66, 0.62, 0.60, 0.59, 0.57, 0.54, 0.55, 0.55, 0.58, 0.57, 0.57, 0.55, 0.56, 0.53, 0.49, 0.46, 0.59, 0.59, 0....

3.编写处理标签数据的工具方法，将N行一列的NDArray转换为N行NumberOfClasses列的NDArray，每行对应的正确分类的值为1，其它列的值为0。这样做是为了向cross-entropy loss公式靠拢

In [3]:
/**
  * 处理标签数据：将N行一列的NDArray转换为N行CLASSES列的NDArray，每行对应的正确分类的值为1，其它列的值为0
  *
  * @param ndArray 标签数据
  * @return N行CLASSES列的NDArray
  */
def makeVectorized(ndArray: INDArray): INDArray = {
  val shape = ndArray.shape()

  val p = Nd4j.zeros(shape(0), CLASSES)
  for (i <- 0 until shape(0)) {
    val double = ndArray.getDouble(i, 0)
    val column = double.toInt
    p.put(i, column, 1)
  }
  p
}

defined function makeVectorized

4.分离和处理图像和标签数据，注意：这里和与SoftmaxLinearClassifier中不同

In [4]:
val test_data = testNDArray.head

val test_expect_result = testNDArray.tail.head
  
val test_p = makeVectorized(test_expect_result)

test_data: INDArray = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0.55, 0.55, 0.56, 0.54, 0.49, 0.45, 0.59, 0.59, 0.62, 0.65, 0.63, 0.62, 0.64, 0.63, 0.64, 0.61, 0.61, 0.62, 0.64, 0.66, 0.67, 0.67, 0.66, 0.62, 0.60, 0.59, 0.57, 0.54, 0.55, 0.55, 0.58, 0.57, 0.57, 0.55, 0.56, 0.53, 0.49, 0.46, 0.59, 0.59, 0....
test_expect_result: INDArray = [3.00, 8.00, 8.00, 0.00, 6.00, 6.00, 1.00, 6.00, 3.00, 1.00, 0.00, 9.00, 5.00, 7.00, 9.00, 8.00, 5.00, 7.00, 8.00, 6.00, 7.00, 0.00, 4.00, 9.00, 5.00, 2.00, 4.00, 0.00, 9.00, 6.00, 6.00, 5.00, 4.00, 5.00, 9.00, 2.00, 4.00, 1.00, 9.00, 5.00, 4.00, 6.00, 5.00, 6.00, 0.00, 9.00, 3.00, 9.00, 7.00, 6.00, 9.00, 8.00, 0.00, 3.00, 8.00, 8.00, 7.00, 7.00, 4.00, 6.00, 7.00, 3.00, 6.00, 3.00, 6.00, 2.00, 1.0...
test_p: INDArray = [[0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
 [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0

5.设置学习率，学习率是Weight变化的快慢的直观描述，学习率设置的过小会导致loss下降的很慢，需要更长时间来训练，学习率设置的过大虽然刚开始下降很快但是会导致在接近最低点的时候在附近徘徊loss下降会非常慢。

In [5]:
implicit val optimizerFactory = new DifferentiableINDArray.OptimizerFactory {
  override def ndArrayOptimizer(
      weight: DifferentiableINDArray.Layers.Weight): L2Regularization = {
    new DifferentiableINDArray.Optimizers.L2Regularization {
      override protected def l2Regularization = 0.03

      var learningRate = 0.001

      override protected def currentLearningRate(): Double = {
        learningRate
      }

      override def updateNDArray(oldValue: INDArray,
                                 delta: INDArray): INDArray = {
        learningRate *= 0.9995
        super.updateNDArray(oldValue, delta)
      }
    }
  }
}

optimizerFactory: AnyRef with OptimizerFactory{def ndArrayOptimizer(weight: com.thoughtworks.deeplearning.DifferentiableINDArray.Layers.Weight): com.thoughtworks.deeplearning.DifferentiableINDArray.Optimizers.L2Regularization} = $sess.cmd4Wrapper$Helper$$anon$2@3141d093

6.编写第一层神经网络

In [6]:
def fullyConnectedThenRelu(inputSize: Int, outputSize: Int)(
    implicit row: From[INDArray]##T): To[INDArray]##T = {
  val w = (Nd4j.randn(inputSize, outputSize) / math.sqrt(outputSize / 2.0)).toWeight * 0.1
  val b = Nd4j.zeros(outputSize).toWeight
  max((row dot w) + b, 0.0)
}

defined function fullyConnectedThenRelu

7.编写softmax函数

In [7]:
def softmax(implicit scores: From[INDArray] ##T): To[INDArray] ##T = {
  val expScores = exp(scores)
  expScores / expScores.sum(1)
}

defined function softmax

8.编写第二层神经网络

In [8]:
def fullyConnectedThenSoftmax(inputSize: Int, outputSize: Int)(
    implicit row: From[INDArray]##T): To[INDArray]##T = {
  val w = (Nd4j.randn(inputSize, outputSize) / math.sqrt(outputSize)).toWeight //* 0.1
  val b = Nd4j.zeros(outputSize).toWeight
  softmax.compose((row dot w) + b)
}

defined function fullyConnectedThenSoftmax

9.将上面的两层神经网络组合起来

In [9]:
def hiddenLayer(implicit input: From[INDArray]##T): To[INDArray]##T = {
  val layer0 = fullyConnectedThenRelu(3072, 500).compose(input)
  fullyConnectedThenSoftmax(500, 10).compose(layer0)
}

val predictor = hiddenLayer

defined function hiddenLayer
predictor: (To[INDArray]{type OutputData = org.nd4j.linalg.api.ndarray.INDArray;type OutputDelta = org.nd4j.linalg.api.ndarray.INDArray;type InputData = org.nd4j.linalg.api.ndarray.INDArray;type InputDelta = org.nd4j.linalg.api.ndarray.INDArray})#T = Compose(Compose(MultiplyINDArray(Exp(Identity()),Reciprocal(Sum(Exp(Identity()),WrappedArray(1)))),PlusINDArray(Dot(Identity(),Weight([[-0.51, -0.22, 0.86, 0.55, -0.05, -0.48, 0.13, 0.13, 0.48, 0.10],
 [-0.25, 0.00, -0.48, 0.32, 0.05, -0.23, -0.27, 0.43, -0.00, 0.22],
 [-0.44, -0.19, 0.19, 0.26, 0.73, -0.18, -0.37, -0.11, -0.12, 0.37],
...

10.编写损失函数Loss Function，将此次判断的结果和真实结果进行计算得出cross-entropy loss并返回

In [10]:
def crossEntropy(
    implicit pair: From[INDArray :: INDArray :: HNil]##T): To[Double]##T = {
  val score = pair.head
  val label = pair.tail.head
  -(label * log(score * 0.9 + 0.1) + (1.0 - label) * log(1.0 - score * 0.9)).sum
}

defined function crossEntropy

11.组合输入层和隐含层

In [11]:
def network(
   implicit pair: From[INDArray :: INDArray :: HNil]##T): To[Double]##T = {
  val input = pair.head
  val label = pair.tail.head
  val score: To[INDArray]##T = predictor.compose(input)
  val hnilLayer: To[HNil]##T = HNil
  crossEntropy.compose(score :: label :: hnilLayer)
}

val trainer = network

defined function network
trainer: (To[Double]{type OutputData = Double;type OutputDelta = Double;type InputData = shapeless.::[org.nd4j.linalg.api.ndarray.INDArray,shapeless.::[org.nd4j.linalg.api.ndarray.INDArray,shapeless.HNil]];type InputDelta = shapeless.:+:[org.nd4j.linalg.api.ndarray.INDArray,shapeless.:+:[org.nd4j.linalg.api.ndarray.INDArray,shapeless.CNil]]})#T = Compose(Negative(ReduceSum(PlusINDArray(MultiplyINDArray(Head(Tail(Identity())),Log(PlusDouble(MultiplyDouble(Head(Identity()),Literal(0.9)),Literal(0.1)))),MultiplyINDArray(PlusDouble(Negative(Head(Tail(Identity()))),Literal(1.0)),Log(PlusDouble(Negative(MultiplyDouble(Head(Identity()),Literal(0.9))),Literal(1.0))))))),HCons(Compose(Compose(Compose(MultiplyINDArray(Exp(Identity()),Reciprocal(Sum(...

12.训练神经网络并观察每次训练loss的变化，loss的变化趋势是降低，但是不是每次都降低(前途是光明的，道路是曲折的)

In [12]:
val lossSeq = for (_ <- 0 until 2000) yield {
  val trainNDArray = ReadCIFAR10ToNDArray.getSGDTrainNDArray(256)
  network.train(
    trainNDArray.head :: makeVectorized(trainNDArray.tail.head) :: HNil)
}

plotly.JupyterScala.init()
val plot = Seq(
  Scatter(
    0 until 2000 by 1,
    lossSeq
  )
)

plot.plot(
  title = "loss on time"
)

lossSeq: Seq[Double] = List(
  684.9188232421875,
  686.76513671875,
  670.3001708984375,
  657.9227294921875,
...
plot: Seq[Scatter] = List(
  Scatter(
    Some(
      Doubles(
        Vector(
...
res11_4: String = "plot-1041040961"

13.使用训练后的神经网络判断测试数据的标签

In [13]:
val result = predictor.predict(test_data)
println(s"result: $result") //输出判断结果

result: [[0.01, 0.08, 0.17, 0.35, 0.02, 0.10, 0.19, 0.00, 0.07, 0.00],
 [0.04, 0.06, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.56, 0.34],
 [0.23, 0.04, 0.00, 0.01, 0.00, 0.00, 0.00, 0.00, 0.64, 0.07],
 [0.37, 0.03, 0.05, 0.01, 0.01, 0.01, 0.00, 0.00, 0.51, 0.01],
 [0.00, 0.00, 0.16, 0.04, 0.59, 0.04, 0.13, 0.03, 0.01, 0.00],
 [0.00, 0.02, 0.05, 0.17, 0.02, 0.09, 0.63, 0.01, 0.00, 0.01],
 [0.00, 0.04, 0.01, 0.68, 0.00, 0.22, 0.06, 0.00, 0.00, 0.00],
 [0.00, 0.00, 0.20, 0.04, 0.38, 0.01, 0.34, 0.03, 0.00, 0.00],
 [0.01, 0.01, 0.30, 0.18, 0.22, 0.20, 0.04, 0.03, 0.02, 0.00],
 [0.02, 0.47, 0.00, 0.01, 0.00, 0.00, 0.00, 0.00, 0.39, 0.11],
 [0.13, 0.00, 0.00, 0.01, 0.00, 0.00, 0.00, 0.00, 0.85, 0.00],
 [0.00, 0.16, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.04, 0.79],
 [0.00, 0.24, 0.07, 0.16, 0.07, 0.16, 0.27, 0.02, 0.01, 0.00],
 [0.04, 0.29, 0.06, 0.20, 0.01, 0.04, 0.26, 0.02, 0.03, 0.03],
 [0.04, 0.25, 0.08, 0.02, 0.02, 0.00, 0.01, 0.11, 0.09, 0.37],
 [0.06, 0.00, 0.18, 0.04, 0.16, 0.03, 0.02, 0.0

result: To.<refinement>.this.type.OutputData = [[0.01, 0.08, 0.17, 0.35, 0.02, 0.10, 0.19, 0.00, 0.07, 0.00],
 [0.04, 0.06, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.56, 0.34],
 [0.23, 0.04, 0.00, 0.01, 0.00, 0.00, 0.00, 0.00, 0.64, 0.07],
 [0.37, 0.03, 0.05, 0.01, 0.01, 0.01, 0.00, 0.00, 0.51, 0.01],
 [0.00, 0.00, 0.16, 0.04, 0.59, 0.04, 0.13, 0.03, 0.01, 0.00],
...

14.编写工具方法，从一行INDArray中获得值最大的元素所在的列，目的是获得神经网络判断的结果，方便和原始标签比较以得出正确率。

In [14]:
/**
  * 从一行INDArray中获得值最大的元素所在的列
  * @param iNDArray
  * @return
  */
def findMaxItemIndex(iNDArray: INDArray): Int = {
  val shape = iNDArray.shape()
  val col = shape(1)
  var maxValue = 0.0
  var maxIndex = 0
  for (index <- 0 until col) {
    val itemValue = iNDArray.getDouble(0, index)
    if (itemValue > maxValue) {
      maxValue = itemValue
      maxIndex = index
    }
  }
  maxIndex
}

defined function findMaxItemIndex

15.判断神经网络对测试数据分类判断的正确率，正确率应该在52%左右。

In [15]:
var right = 0

val shape = result.shape()
for (row <- 0 until shape(0)) {
  val rowItem = result.getRow(row)
  val index = findMaxItemIndex(rowItem)
  if (index == test_expect_result.getDouble(row, 0)) {
    right += 1
  }
}
println(s"the result is $right %")

the result is 47 %


right: Int = 47
shape: Array[Int] = Array(100, 10)


16.[完整代码](https://github.com/izhangzhihao/deeplearning-tutorial/blob/master/src/main/scala/com/thoughtworks/deeplearning/tutorial/TwoLayerNet.scala)